# Libs & Settings

In [1]:
import numpy as np
import polars as pl
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import plotly.express as px
import matplotlib.pyplot as plt
from typing import List
import lightgbm as lgb
from mlforecast import MLForecast
from dateutil.relativedelta import relativedelta
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [2]:
%run -i "/Users/ivanandrusin/Desktop/PyCharmProjects/data-wagon-pioneers/.venv/lib/python3.9/site-packages/hierarchicalforecast/utils.py"

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from statsforecast import StatsForecast
from sklearn.metrics import mean_squared_error
from statsforecast.models import (
    AutoARIMA,
    # HistoricAvarage,
    SeasonalNaive,
    HoltWinters,
    MSTL,
)

from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, TopDown, ERM, OptimalCombination

In [4]:
data_folder = Path().cwd().parent / 'data'

# Data Analysis

In [5]:
# Read data.
df = pd.read_csv(
    data_folder / "fact_train_test.csv", 
    sep=";", 
    decimal=",", 
    encoding="windows-1251"
)
# Convert
df["period"] = df["period"].astype("datetime64[ns]")

In [6]:
# Count uniques.
df.nunique()

period                       85
rps                           2
podrod                        7
filial                        3
client_sap_id              2036
freight_id                 2652
sender_station_id          3256
recipient_station_id       4684
sender_organisation_id    33758
real_weight               50461
real_wagon_count           3669
dtype: int64

## Aggregate TS

In [7]:
def process_ts(df, uid, target=['real_wagon_count']):
    grp = ['period'] + uid
    df_new = (
        df[grp + target]
        .query('period >= "2017-01-01"')
        .groupby(grp)[target]
        .sum()
        .reset_index()
    )
    pdf = pl.from_pandas(df_new)
    ddf = (
        pdf.with_columns(
            pdf.select([
                pl.concat_str(pl.col(uid), separator='_')
                .alias("unique_ts"),
            ])
        )
        .to_pandas()
    )
    return ddf

In [8]:
general_agg = [
    'rps', 
    'client_sap_id',
    'sender_station_id',
    'recipient_station_id'
]

In [9]:
df_agg = process_ts(df=df, uid=general_agg)

## Mappings

In [7]:
def add_master_data_mappings(df: pd.DataFrame) -> pd.DataFrame:
    # = Пути к справочникам - откорректировать если в реальной системе будут лежать по другому адресу =
    client_mapping_file = data_folder / "client_mapping.csv"
    freight_mapping_file = data_folder / "freight_mapping.csv"
    station_mapping_file = data_folder / "station_mapping.csv"

    # Клиент - холдинг
    client_mapping = pd.read_csv(
        client_mapping_file,
        sep=";",
        decimal=",",
        encoding="windows-1251",
    )
    df = pd.merge(df, client_mapping, how="left", on="client_sap_id")

    # Груз
    freight_mapping = pd.read_csv(
        freight_mapping_file, sep=";", decimal=",", encoding="windows-1251"
    )
    df = pd.merge(df, freight_mapping, how="left", on="freight_id")

    # Станции
    station_mapping = pd.read_csv(
        station_mapping_file,
        sep=";",
        decimal=",",
        encoding="windows-1251",
    )
    df = pd.merge(
        df,
        station_mapping.add_prefix("sender_"),
        how="left",
        on="sender_station_id",
    )
    df = pd.merge(
        df,
        station_mapping.add_prefix("recipient_"),
        how="left",
        on="recipient_station_id",
    )

    return df


# Preprocess data.
def process_ts(df, grp, target='real_wagon_count', cutoff="20120101"):
    _df = (
        df[grp + [target]]
        .query(f'period >= {cutoff}')
        .groupby(grp)[target]
        .sum()
        .reset_index()
    )
    return _df

# Generate unique id for initial data.
def get_unique_id(df, uid):
    pdf = pl.from_pandas(df)
    _df = (
        pdf.with_columns(
            pdf.select([
                pl.concat_str(
                    pl.col(uid), 
                    separator='_'
                )
                .alias("unique_ts"),
            ]))
        .to_pandas()
    )
    return _df

In [8]:
# Map id columns.
df_mp = add_master_data_mappings(df)
# Time series aggregation levels.
accuracy_granularity = [
    "period",
    "rps",
    "holding_name",
    "sender_department_name",
    "recipient_department_name",
]

df_mp_agg = (
    process_ts(df=df_mp, grp=accuracy_granularity, cutoff="20170101")
    .set_index("period")
    .astype(np.int64)
    .reset_index()
)
# Check uniques.
df_mp_agg.nunique()

period                         75
rps                             2
holding_name                 1892
sender_department_name        133
recipient_department_name     143
real_wagon_count             5621
dtype: int64

In [9]:
uid = [
    'rps', 
    'podrod',
    'filial',
    'client_sap_id',
    'freight_id',
    'sender_station_id',
    'recipient_station_id',
    'sender_organisation_id'
]

df_mp_uid = get_unique_id(df=df_mp, uid=uid)

In [10]:
df_mp_agg_ = get_unique_id(df_mp_agg, accuracy_granularity[1:]).drop(columns=accuracy_granularity[1:])
cartesian = pd.DataFrame(
    data=pd.MultiIndex.from_product(
        [df_mp_agg_.period.unique(), 
         df_mp_agg_.unique_ts.unique()]
    ).to_list(), 
    columns=['period', 'unique_ts']
)

In [11]:
df_mp_agg_full = cartesian.merge(df_mp_agg_, how='left', on = ['period', 'unique_ts']).fillna(0)

In [12]:
df_mp_agg_full[accuracy_granularity[1:]] = (
    df_mp_agg_full['unique_ts']
    .str
    .split('_', n=len(accuracy_granularity[1:]), expand=True)
)

In [13]:
df_mp_agg_full = (
    df_mp_agg_full.drop(columns=['unique_ts'])
    .assign(real_wagon_count=lambda x: x['real_wagon_count'].astype(np.int64))
)

## Forecast

In [20]:
# Cut-off date.
SPLIT_DATE = df_mp_agg['period'].max() - relativedelta(months=5)

In [21]:
hf = df_mp_agg_full.rename(columns={'period': 'ds', 'real_wagon_count': 'y'})
hf[
    ["rps", "holding_name", "sender_department_name", "recipient_department_name"]
] = hf[
    ["rps", "holding_name", "sender_department_name", "recipient_department_name"]
].astype(str)

In [19]:
#get train and validation datasets
train = hf.loc[hf['ds'] <= SPLIT_DATE]
valid = hf.loc[hf['ds'] > SPLIT_DATE]
#get horizon value
# h = valid['ds'].nunique()

NameError: name 'SPLIT_DATE' is not defined

In [22]:
train = hf.copy()

In [23]:
train

,ds,y,rps,holding_name,sender_department_name,recipient_department_name
0,2017-01-01,1,0,10,99,98
1,2017-01-01,3,0,10,99,99
2,2017-01-01,4,0,10,99,101
3,2017-01-01,10,0,10,133,2
4,2017-01-01,2,0,10,133,3
...,...,...,...,...,...,...
4658470,2023-03-01,10,1,2028,161,136
4658471,2023-03-01,2,1,2071,43,148
4658472,2023-03-01,164,1,2076,110,110
4658473,2023-03-01,3,1,2140,55,154


In [24]:
#create a list of the columns that represent the different levels of the hierarchy
spec = [
    ['rps'],
    ['rps', 'holding_name'],
    ['rps', 'holding_name', 'sender_department_name'],
    ['rps', 'holding_name', 'sender_department_name', 'recipient_department_name']
]

In [25]:
# Split the data into different levels
train_agg, S_train, tags = aggregate(train, spec)
# valid_agg, _, _ = aggregate(valid, spec)

In [26]:
@njit
def rolling_mean_3(x):
    return rolling_mean(x, window_size=3)

## ML models

In [27]:
mlf_models = [
    lgb.LGBMRegressor(verbosity=-1),
    xgb.XGBRegressor(),
    RandomForestRegressor(random_state=0),
]

mlf = MLForecast(
    models=mlf_models,
    freq='MS',
    lag_transforms={
        2: [expanding_mean],
        2: [rolling_mean_3]
    },
    lags=[1, 2]
)

mlf.fit(df=train_agg.reset_index())

MLForecast(models=[LGBMRegressor, XGBRegressor, RandomForestRegressor], freq=<MonthBegin>, lag_features=['lag1', 'lag2', 'rolling_mean_3_lag2'], date_features=[], num_threads=1)

In [28]:
# Generate predictions.
base_forecast_df = mlf.predict(5)

In [30]:
# Forecast validation.
res = valid_agg.reset_index().merge(base_forecast_df, how='left', on=['unique_id', 'ds']).fillna(0)
model_cols = [col for col in base_forecast_df.columns if col not in ['unique_id', 'ds']]
for col in model_cols:
    print(
        f"{col}: {mean_squared_error(res['y'], res[col], squared=False)/1e3}")

LGBMRegressor: 2.3175828771954503
XGBRegressor: 2.595827261500487
RandomForestRegressor: 2.5959406283036093


In [31]:
# reconcilers = [
#     BottomUp(),
#     # MinTrace('ols')
# ]
# hrec = HierarchicalReconciliation(reconcilers=reconcilers)
# Y_rec_mf = hrec.reconcile(Y_hat_df=base_forecast_df.set_index('unique_id'), Y_df = train_agg, S=S_train, tags=tags)

In [29]:
answer = base_forecast_df.copy()
answer[accuracy_granularity[1:]] = (
    answer['unique_id']
    .str
    .split('/', n=len(accuracy_granularity[1:]), expand=True)
)
answer = answer.dropna().reset_index(drop=True)

In [30]:
answer.head()

,unique_id,ds,LGBMRegressor,XGBRegressor,RandomForestRegressor,rps,holding_name,sender_department_name,recipient_department_name
0,0/10/133/1,2023-04-01,0.419614,0.589790,0.578435,0,10,133,1
1,0/10/133/1,2023-05-01,0.766676,0.589790,0.937090,0,10,133,1
2,0/10/133/1,2023-06-01,0.766676,0.589790,0.809849,0,10,133,1
3,0/10/133/1,2023-07-01,0.766676,0.589790,0.809849,0,10,133,1
4,0/10/133/1,2023-08-01,0.859948,0.378985,0.809849,0,10,133,1


In [31]:
df_mp_uid_full = (
    df_mp_uid[
    (df_mp_uid['period'] >= '2023-01-01') 
    & (df_mp_uid.client_sap_id != -1)]
    .reset_index(drop=True)
    .rename(columns={'unique_ts': 'unique_ts_full'})
)
df_mp_uid_full.holding_name = df_mp_uid_full.holding_name.astype(np.int64)

In [32]:
df_mp_uid_2 = get_unique_id(df=df_mp_uid_full, uid=accuracy_granularity[1:])

In [33]:
df_calc = df_mp_uid_2.groupby(['unique_ts', 'unique_ts_full']).unique_ts_full.count().to_frame().rename(columns={'unique_ts_full': 'count'}).reset_index()

In [34]:
df_calc['tot_count'] = df_calc.groupby('unique_ts')['count'].transform('sum')

In [35]:
df_calc['perc'] = df_calc['count']/df_calc['tot_count']

In [36]:
df_calc.head()

,unique_ts,unique_ts_full,count,tot_count,perc
0,0_1001_86_43,0_1_2_383_1087_26726_33095_27539,1,1,1.0
1,0_1001_86_75,0_0_1_383_1087_26726_12872_27539,1,1,1.0
2,0_1001_86_9,0_0_1_383_1087_26726_35100_27539,1,1,1.0
3,0_1001_86_97,0_0_1_383_1087_26726_11053_27539,1,1,1.0
4,0_1003_8_136,0_0_1_1795_3297_36446_38316_28073,1,1,1.0


In [37]:
df_calc[accuracy_granularity[1:]] = (
    df_calc['unique_ts']
    .str
    .split('_', n=len(accuracy_granularity[1:]), expand=True)
)

In [38]:
sub_answer = answer.merge(df_calc[
    ['rps', 'holding_name', 'sender_department_name', 'recipient_department_name', 'perc', 'unique_ts_full']
], how='left', on=['rps', 'holding_name', 'sender_department_name', 'recipient_department_name']
).dropna().reset_index(drop=True)

In [39]:
sub_answer.head()

,unique_id,ds,LGBMRegressor,XGBRegressor,RandomForestRegressor,rps,holding_name,sender_department_name,recipient_department_name,perc,unique_ts_full
0,0/1001/86/43,2023-04-01,0.419614,0.589790,0.578435,0,1001,86,43,1.0,0_1_2_383_1087_26726_33095_27539
1,0/1001/86/43,2023-05-01,0.766676,0.589790,0.937090,0,1001,86,43,1.0,0_1_2_383_1087_26726_33095_27539
2,0/1001/86/43,2023-06-01,0.766676,0.589790,0.809849,0,1001,86,43,1.0,0_1_2_383_1087_26726_33095_27539
3,0/1001/86/43,2023-07-01,0.766676,0.589790,0.809849,0,1001,86,43,1.0,0_1_2_383_1087_26726_33095_27539
4,0/1001/86/43,2023-08-01,0.859948,0.378985,0.809849,0,1001,86,43,1.0,0_1_2_383_1087_26726_33095_27539


In [40]:
sub_answer_lgb = sub_answer[['ds', 'LGBMRegressor', 'perc', 'unique_ts_full']]

In [41]:
sub_answer_lgb['forecast_wagon_count'] = sub_answer_lgb['LGBMRegressor'] * sub_answer_lgb['perc']
sub_answer_lgb['forecast_wagon_count'] = sub_answer_lgb['forecast_wagon_count'].round().astype(np.int64)

In [42]:
sub_answer_lgb = sub_answer_lgb.rename(columns={'ds': 'period'}).drop(columns=['LGBMRegressor', 'perc'])

In [43]:
sub_answer_lgb[uid] = (
    sub_answer_lgb['unique_ts_full']
    .str
    .split('_', n=len(uid), expand=True)
)

In [44]:
sub_answer_lgb = sub_answer_lgb.drop(columns=['unique_ts_full'])[['period'] + uid + ['forecast_wagon_count']]

In [45]:
sub_answer_lgb['forecast_weight'] = sub_answer_lgb['forecast_wagon_count']*60

In [48]:
sub_answer_lgb['forecast_weight'] = sub_answer_lgb['forecast_weight'].astype(np.float64)

In [51]:
right_order = [
    'period',
    'rps',
    'podrod',
    'filial',
    'client_sap_id',
    'freight_id',
    'sender_station_id',
    'recipient_station_id',
    'sender_organisation_id',
    'forecast_weight',
    'forecast_wagon_count'
]

In [52]:
sub_answer_lgb = sub_answer_lgb[right_order]

In [54]:
sub_answer_lgb.to_csv(
    data_folder / 'base' / "forecast_lgb.csv", 
    index=False, sep=";", 
    decimal=",", 
    encoding="windows-1251"
)

In [53]:
sub_answer_lgb

,period,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,forecast_weight,forecast_wagon_count
0,2023-04-01,0,1,2,383,1087,26726,33095,27539,0.0,0
1,2023-05-01,0,1,2,383,1087,26726,33095,27539,60.0,1
2,2023-06-01,0,1,2,383,1087,26726,33095,27539,60.0,1
3,2023-07-01,0,1,2,383,1087,26726,33095,27539,60.0,1
4,2023-08-01,0,1,2,383,1087,26726,33095,27539,60.0,1
...,...,...,...,...,...,...,...,...,...,...,...
298970,2023-08-01,1,5,1,431,1649,8812,34400,26775,180.0,3
298971,2023-08-01,1,5,1,431,4293,11385,34400,28632,360.0,6
298972,2023-08-01,1,5,1,431,4293,11421,34400,28130,540.0,9
298973,2023-08-01,1,5,1,431,4293,8893,34400,31153,540.0,9
